In [ ]:
import cv2
import numpy as np
import os


# Capture Images

In [ ]:
cam=cv2.VideoCapture(0)
i=1
while True:
    outfr,captur=cam.read()
    if outfr==False:
        break    
    cv2.imshow('Output',captur)
    if cv2.waitKey(1)==13:
        break
    elif cv2.waitKey(32)==ord(' '):
        cv2.imwrite(str(i)+'.jpg',captur)
        i+=1
        print('Captured')
    else:
        continue
cv2.destroyAllWindows()

cam.release()

In [ ]:
cube_sz=25   
pointob = np.zeros((6*9,3), np.float32)
pointob[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)*cube_sz

In [ ]:
objarr=[]  
imgarr=[] 

# Identify Edges of Chessboard

In [ ]:
def identify_edge(image):
    gray=cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
    ret,edge=cv2.findChessboardCorners(gray,(9,6),None)
    if ret == True:
        objarr.append(pointob)
        imgarr.append(edge)
        img = cv2.drawChessboardCorners(image, (9,6), edge,ret)
        return img
    else:
        return image

# Through Images

In [ ]:
fl_img=os.listdir('images')
path=os.getcwd()
for i in fl_img:
    path1=os.path.join(path,'images',i)
    image=cv2.imread(path1)
    im=identify_edge(image)
    cv2.imshow('out',image)
    cv2.waitKey(0)
cv2.destroyAllWindows()

# Through Video

In [ ]:
cam=cv2.VideoCapture(0)
while True:
    outfr,captur=cam.read()
    if outfr==False:
        break    
    im=identify_edge(captur)
    cv2.imshow('Output',im)
    if cv2.waitKey(1)==13:
        break
cv2.destroyAllWindows()
cam.release()

# Camera Calibration

In [ ]:
shp_img=(640,480)
ret, metrx, dstn, rotavec, transvec = cv2.calibrateCamera(objarr, imgarr,shp_img,None,None)

In [ ]:
np.savez('camera_matrix',metrx=metrx,dstn=dstn)

In [ ]:
with np.load('camera_matrix.npz') as file:
    metrx,dstn=[file[i] for i in ['metrx','dstn']]
    

In [ ]:
pointob = np.zeros((6*9,3), np.float32)
pointob[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

In [ ]:
axis = np.float32([[3,0,0], [0,3,0], [0,0,-3]]).reshape(-1,3)    
 

In [ ]:
axis = np.float32([[0,0,0], [0,3,0], [3,3,0], [3,0,0],
                   [0,0,-3],[0,3,-3],[3,3,-3],[3,0,-3] ])    


# Draw Cube

In [ ]:
def cube_build(img, edge, imgpts):
    imgpts = np.int32(imgpts).reshape(-1,2)
    img = cv2.drawContours(img, [imgpts[:4]],-1,(0,255,0),3)
    for i,j in zip(range(4),range(4,8)):
        img = cv2.line(img, tuple(imgpts[i]), tuple(imgpts[j]),(255),3)
    img = cv2.drawContours(img, [imgpts[4:]],-1,(0,0,255),3)
    return img

In [ ]:
def construct(img, edge, imgpts):
    corner = tuple(edge[0].ravel())
    img = cv2.line(img, corner, tuple(imgpts[0].ravel()), (255,0,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[1].ravel()), (0,255,0), 5)
    img = cv2.line(img, corner, tuple(imgpts[2].ravel()), (0,0,255), 5)
    return img

In [ ]:
path='images/'
root=os.getcwd()

In [ ]:
l=os.listdir(path)
for i in l:
    path1=os.path.join(root,path,i)
    im=cv2.imread(path1)
    gray=cv2.cvtColor(im,cv2.COLOR_BGR2GRAY)
    ret,edge = cv2.findChessboardCorners(gray,(9,6),None)
    if ret == True:
        _,rotavec,transvec,_=cv2.solvePnPRansac(pointob,edge,metrx,dstn)
        imgpts,_=cv2.projectPoints(axis,rotavec,transvec,metrx,dstn)
        img = construct(im,edge,imgpts)
        cv2_imshow(img)
        cv2.waitKey()
        cv2.destroyAllWindows()

# Cube in Video

In [ ]:
cam=cv2.VideoCapture(0)
out = cv2.VideoWriter('output.avi',cv2.VideoWriter_fourcc('M','J','P','G'), 10, (int(cam.get(3)),int(cam.get(4))))
while True:
    outfr,captur=cam.read()
    if outfr==False:
        break
    gray=cv2.cvtColor(captur,cv2.COLOR_BGR2GRAY)
    ret,edge = cv2.findChessboardCorners(gray,(9,6),None)
    if ret == True:
        _,rotavec,transvec,_=cv2.solvePnPRansac(pointob,edge,metrx,dstn)
        imgpts,_=cv2.projectPoints(axis,rotavec,transvec,metrx,dstn)
        captur = cube_build(captur,edge,imgpts)
    cv2.imshow('images',captur)
    out.write(captur)
    if cv2.waitKey(1)==13:
        break
        
cv2.destroyAllWindows()
cam.release()
